In [1]:
# dataset: https://www.dropbox.com/scl/fo/nz1j92xpr6eet3fa5mx5i/AJf7Zr3-So7jmv6qHk9lpik/v2?dl=0&rlkey=tem27v1d2raf2nnlcq6cd49ev&subfolder_nav_tracking=1

# Using Swarm Alpha satellite data available by the warmup dataset, find the average atmospheric density measured between 400km and 450km altitude during in 2015. 

In [ ]:
# Q3

import pandas as pd
from glob import glob
import pathlib
import os 

os.chdir("../data_local/STORM-AI/warmup/v2/")

# ---- 1.  load altitude for every File ID -----------------------------
init_path   = "wu001_to_wu715-initial_states.csv"  # adjust
init_df     = (
    pd.read_csv(init_path, parse_dates=["Timestamp"])            # Altitude column lives here
      .loc[:, ["File ID", "Altitude (km)"]]
      .rename(columns={"File ID": "file_id", "Altitude (km)": "alt_km"})
)

# ---- 2.  collect every Swarm‑Alpha density slice for 2015 ------------
dens_frames = []
for f in glob("Sat_Density/swarma-*2015*.csv"):                # adjust pattern
    df = pd.read_csv(f, parse_dates=["Timestamp"])
    df = df[df.Timestamp.dt.year == 2015]                         # keep 2015 only
    if df.empty:                                  # skip 3‑day files outside 2015
        continue

    # File‑ID is the middle token in the name: SWARM_A-<ID>-YYYYmmDD_to_*.csv
    file_id = pathlib.Path(f).stem.split("-")[1]
    df["file_id"] = file_id
    dens_frames.append(df[["Timestamp", "file_id", "Orbit Mean Density (kg/m^3)"]])

dens_df = pd.concat(dens_frames, ignore_index=True)

# ---- 3.  join altitude → filter 450‑500 km → average -----------------
merged   = dens_df.merge(init_df, on="file_id", how="left")
slice_   = merged[merged.alt_km.between(450, 500)]
mean_rho = slice_["Orbit Mean Density (kg/m^3)"].mean()

print(f"Swarm‑Alpha mean density (450–500 km, 2015): {mean_rho:.3e}  kg/m³")


Swarm‑Alpha mean density (450–500 km, 2015): 8.593e+30  kg/m³
